In [1]:
!pip install kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from IPython.display import clear_output
clear_output()

In [2]:
# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

In [3]:

data = pd.read_csv(path + "/creditcard.csv")

data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


El 0,1% de los datos es de la clase positiva (fraude)

In [4]:
print("Numero de filas y columnas:", data.shape)
print(data.groupby('Class').size()/len(data))

Numero de filas y columnas: (284807, 31)
Class
0    0.998273
1    0.001727
dtype: float64


# Modelado

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['Class']), 
                                                    data['Class'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [12]:
def cross_validation(model, x,y,folds):
    from sklearn.model_selection import KFold
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import roc_auc_score
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    accs = []
    recs = []
    precs = []
    f1s = []
    rocs = []
    stats = []
    iters = []
    for train_index, test_index in kf.split(x):
        x_train, x_val = x.iloc[train_index], x.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_val)

        # accuracy
        accs.append(accuracy_score(y_val, y_pred))
        # precision
        precs.append(precision_score(y_val, y_pred))
        # recall
        recs.append(recall_score(y_val, y_pred))
        # f1-score
        f1s.append(f1_score(y_val, y_pred))
        # AUC
        rocs.append(roc_auc_score(y_val, y_pred))

        #status del modelo
        if isinstance(model,LogisticRegression):
            status =  model.n_iter_[0] < model.max_iter
            iterations = model.n_iter_[0]
            stats.append(status)
            iters.append(iterations)

        # 

    return pd.DataFrame(
        {
            "model": [type(model) for i in range(folds)],
            "accuracy": accs,
            "precision": precs,
            "recall": recs,
            "f1": f1s,
            "roc_auc": rocs,
            "convergence": stats,
            "iterations": iters
        }
    )
    


In [ ]:
cross_validation(LogisticRegression(max_iter=1000), 
                data.drop(columns=["Class"]),
                data["Class"],
                folds = 5)